In [7]:
pass


In [8]:
# folder = 'books'
# novel_name='Hare'

In [9]:
# ! python -m nltk.downloader punkt


In [10]:
# ! pip install spacy pandas numpy networkx matplotlib afinn nltk scikit-learn

In [11]:
# ! pip install spacy networkx matplotlib nltk
# ! python -m spacy download en_core_web_sm


In [12]:
import os
import re
import spacy
import json
import codecs
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
from collections import Counter, defaultdict
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from itertools import combinations
from itertools import permutations

ModuleNotFoundError: No module named 'spacy'

In [ ]:
# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

In [ ]:
# # Define relationship keywords
# relationship_keywords = {
#     'father': ['father of', 'his daughter', 'her father', 'dad of', 'is the father of'],
#     'mother': ['mother of', 'his son', 'her daughter', 'mom of', 'is the mother of'],
#     'daughter': ['daughter of', 'is the daughter of'],
#     'son': ['son of', 'is the son of'],
#     'wife': ['wife of', 'his wife', 'is the wife of'],
#     'husband': ['husband of', 'her husband', 'is the husband of'],
#     'friend': ['friend of', 'his friend', 'her friend', 'is a friend of'],
#     'sister': ['sister of', 'is the sister of'],
#     'brother': ['brother of', 'is the brother of'],
#     'lover': ['lover of', 'in love with', 'is in love with'],
#     'partner': ['partner of', 'is the partner of'],
#     'colleague': ['colleague of', 'is a colleague of'],
# }

relationship_keywords = {
    
    # Immediate Family
    "father": [
        "father of", "his daughter", "her father", "dad of", "is the father of",
        "paternal", "biological father", "stepfather", "adoptive father", "foster father",
        "father to", "dad to", "parent of"
    ],
    "mother": [
        "mother of", "his son", "her daughter", "mom of", "is the mother of",
        "maternal", "biological mother", "stepmother", "adoptive mother", "foster mother",
        "mother to", "mom to", "parent of"
    ],
    "daughter": [
        "daughter of", "is the daughter of", "his daughter", "her daughter",
        "biological daughter", "stepdaughter", "adopted daughter", "foster daughter",
        "daughter to", "child of", "girl child of"
    ],
    "son": [
        "son of", "is the son of", "his son", "her son",
        "biological son", "stepson", "adopted son", "foster son",
        "son to", "child of", "boy child of"
    ],
    "wife": [
        "wife of", "his wife", "is the wife of","wives", "married to", "marry",
        "spouse of", "lawfully wedded wife", "ex-wife", "former wife",
        "wife to", "wedded to", "bride of"
    ],
    "husband": [
        "husband of", "her husband", "is the husband of", "married to", "marry",
        "spouse of", "lawfully wedded husband", "ex-husband", "former husband",
        "husband to", "wedded to","marry", "groom of"
    ],
    "sister": [
        "sister of", "is the sister of", "his sister", "her sister",
        "biological sister", "stepsister", "half-sister", "twin sister",
        "younger sister","mother's daughter", "father's daughter", "older sister","sister's sister", "sibling sister"
    ],
    "brother": [
        "brother of", "is the brother of", "his brother", "her brother",
        "biological brother", "stepbrother", "half-brother", "twin brother",
        "younger brother", "older brother","sister's brother"," brother's brother", "sibling brother"
    ],

    # Extended Family (In-Laws)
    "father-in-law": [
        "father-in-law of", "is the father-in-law of", "his father-in-law", "her father-in-law",
        "husband's father", "wife's father", "spouse's father", "father of spouse"
    ],
    "mother-in-law": [
        "mother-in-law of", "is the mother-in-law of", "his mother-in-law", "her mother-in-law",
        "husband's mother", "wife's mother", "spouse's mother", "mother of spouse"
    ],
    "son-in-law": [
        "son-in-law of", "is the son-in-law of", "his son-in-law", "her son-in-law",
        "daughter's husband", "child's husband", "spouse's son", "husband of daughter"
    ],
    "daughter-in-law": [
        "daughter-in-law of", "is the daughter-in-law of", "his daughter-in-law", "her daughter-in-law",
        "son's wife", "child's wife", "spouse's daughter", "wife of son"
    ],
    "brother-in-law": [
        "brother-in-law of", "is the brother-in-law of", "his brother-in-law", "her brother-in-law",
        "husband's brother", "wife's brother", "sister's husband", "spouse's brother",
        "brother of spouse"
    ],
    "sister-in-law": [
        "sister-in-law of", "is the sister-in-law of", "his sister-in-law", "her sister-in-law",
        "husband's sister", "wife's sister", "brother's wife", "spouse's sister",
        "sister of spouse"
    ],

    # Other Relationships
    "friend": [
        "friend of", "his friend", "her friend", "is a friend of",
        "close friend", "best friend", "childhood friend", "school friend",
        "work friend", "mutual friend","friend's friend", "friends with", "pal of", "buddy of"
    ],
    "lover": [
        "lover of", "in love with", "is in love with", "romantic partner",
        "significant other", "sweetheart", "beloved", "paramour", "flame",
        "romantically involved with", "dating", "seeing"
    ],
    "partner": [
        "partner of", "is the partner of", "life partner", "domestic partner",
        "business partner", "work partner", "collaborative partner",
        "cohabiting partner", "unmarried partner", "common-law partner"
    ],
    "colleague": [
        "colleague of", "is a colleague of", "coworker of", "workmate of",
        "fellow employee", "office mate", "professional associate",
        "team member", "peer at work", "co-worker"
    ],
    "uncle": [
        "uncle of", "is the uncle of", "his uncle", "her uncle",
        "paternal uncle", "maternal uncle", "uncle by marriage",
        "father's brother", "mother's brother"
    ],
    "aunt": [
        "aunt of", "is the aunt of", "his aunt", "her aunt",
        "paternal aunt", "maternal aunt", "aunt by marriage",
        "father's sister", "mother's sister"
    ],
    "cousin": [
        "cousin of", "is the cousin of", "his cousin", "her cousin",
        "first cousin", "second cousin", "distant cousin",
        "cousin on father's side", "cousin on mother's side"
    ],
    "grandfather": [
        "grandfather of", "is the grandfather of", "his grandfather", "her grandfather",
        "paternal grandfather", "maternal grandfather", "grandpa of", "granddad of"
    ],
    "grandmother": [
        "grandmother of", "is the grandmother of", "his grandmother", "her grandmother",
        "paternal grandmother", "maternal grandmother", "grandma of", "granny of"
    ],
    "grandson": [
        "grandson of", "is the grandson of", "his grandson", "her grandson",
        "grandchild", "son's son", "daughter's son"
    ],
    "granddaughter": [
        "granddaughter of", "is the granddaughter of", "his granddaughter", "her granddaughter",
        "grandchild", "son's daughter", "daughter's daughter"
    ],
    "nephew": [
        "nephew of", "is the nephew of", "his nephew", "her nephew",
        "brother's son", "sister's son", "son of sibling"
    ],
    "niece": [
        "niece of", "is the niece of", "his niece", "her niece",
        "brother's daughter", "sister's daughter", "daughter of sibling"
    ],
    "boss": [
        "boss of", "is the boss of", "supervisor of", "manager of",
        "employer of", "superior of", "direct supervisor"
    ],
    "employee": [
        "employee of", "is the employee of", "subordinate of", "staff of",
        "worker of", "underling of", "team member of"
    ],
    "mentor": [
        "mentor of", "is the mentor of", "advisor of", "guide of",
        "teacher of", "coach of", "tutor of"
    ],
}


In [ ]:
# Function to load novels from a folder
def load_novel(book_name, path):
    book_list = os.listdir(path)
    book_list = [i for i in book_list if i.find(book_name) >= 0]
    novel = ''
    for i in book_list:
        with codecs.open(path / i, 'r', encoding='utf-8', errors='ignore') as f:
            data = f.read().replace('\r', ' ').replace('\n', ' ').replace("\'", "'")
        novel += ' ' + data
    return novel

In [ ]:
# Function to extract character names from a sentence
def extract_names(sentence):
    doc = nlp(sentence)
    name_entity = [str(x).lower().replace("'s", "") for x in doc.ents if x.label_ == 'PERSON']
    return list(set(name_entity))  # Remove duplicates


In [ ]:
# Function to refine character names
def refine_character_names(ner_results, text, common_words_set, min_freq=1):
    name_freq = Counter(ner_results)
    filtered_names = {name for name, freq in name_freq.items() if freq >= min_freq}
    refined_names = {name for name in filtered_names if name.lower() not in common_words_set}
    return list(refined_names)


In [ ]:
# Function to extract frequently occurring names from a novel
def iterative_name_extraction(sentence_list, threshold_rate=0.0005):
    output = []
    for sentence in sentence_list:
        name_list = extract_names(sentence)
        if name_list:
            output.append(name_list)
    output = [name for sublist in output for name in sublist]  # Flatten the list
    output = Counter(output)
    return [x for x in output if output[x] >= threshold_rate * len(sentence_list)]


In [ ]:
# Function to extract relationships from text
def extract_relationships_from_text(text, characters):
    relationships = defaultdict(list)
    sentences = [sent.text for sent in nlp(text).sents]

    for sentence in sentences:
        sent_lower = sentence.lower()
        for char1, char2 in combinations(characters, 2):
            if char1.lower() in sent_lower and char2.lower() in sent_lower:
                found_relation = False
                for relation, patterns in relationship_keywords.items():
                    if any(p in sent_lower for p in patterns):
                        relationships[(char1, char2)].append((relation, sentence))
                        found_relation = True
                        break
                # If no specific relation found, check for pronouns
                if not found_relation:
                    for pronoun in ['he', 'she', 'his', 'her', 'whose']:
                        if pronoun in sent_lower:
                            if char1.lower() in sent_lower and pronoun in sent_lower:
                                relationships[(char1, char2)].append(('refers to', sentence))
                            elif char2.lower() in sent_lower and pronoun in sent_lower:
                                relationships[(char2, char1)].append(('refers to', sentence))
    return relationships

In [ ]:
# Function to filter out place names
def filter_place_names(entities):
    place_names = {ent.text.lower() for ent in nlp(" ".join(entities)).ents if ent.label_ == "GPE"}
    return [name for name in entities if name.lower() not in place_names]


In [ ]:
# Function to normalize character names
def normalize_names(names):
    normalized = {}
    for name in names:
        if name in normalized:
            continue
        # Check for variations (e.g., "jesus" and "jesus christ")
        normalized[name] = name
        for other_name in names:
            if name in other_name or other_name in name:
                normalized[other_name] = name
    return normalized

In [ ]:
# Function to create the graph
def create_graph(name_list, relationships, plt_name):
    G = nx.Graph()
    G.add_nodes_from(name_list)

    for (char1, char2), rel_list in relationships.items():
        if rel_list:
            dominant_relation = rel_list[0][0]  # Get the first relation as dominant
            G.add_edge(char1, char2, relation=dominant_relation)

    pos = nx.circular_layout(G)
    edge_labels = {(char1, char2): G[char1][char2]['relation'] for char1, char2 in G.edges()}

    plt.figure(figsize=(20, 20))
    nx.draw(G, pos, with_labels=True, node_color='#A0CBE2', node_size=3000, font_size=15)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='darkred', font_size=12)
    plt.title(plt_name)
    plt.savefig(f"{plt_name}.png")  # Save the graph as a PNG file
    plt.show()

In [ ]:
# Main processing logic
folder = 'books'
novel_name = 'Ends With Us'
novel_folder = Path(os.getcwd()) / folder
novel = load_novel(novel_name, novel_folder)
sentence_list = sent_tokenize(novel)

# Extract character names
preliminary_name_list = iterative_name_extraction(sentence_list)

# Define your common words list here
words = []  # You can populate this list with common words to filter out

refined_names = refine_character_names(preliminary_name_list, novel, words)
filtered_names = filter_place_names(refined_names)

# Normalize names to handle variations
normalized_names = normalize_names(filtered_names)

# Extract relationships
relationships = extract_relationships_from_text(novel, normalized_names)

# Create and display the graph
create_graph(normalized_names.values(), relationships, f"{novel_name} Relationship Graph")